In [72]:
import pandas as pd
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
# from sklearn.metrics import r2_score
# from matplotlib import cm
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import seaborn.objects as so

In [73]:
fn1 = './data/bronx.csv'
fn2 = './data/brooklyn.csv'
fn3 = './data/manhatten.csv'
fn4 = './data/queens.csv'
fn5 = './data/staten_island.csv'
fn6 = './data/paris.csv'

bronx = pd.read_csv(fn1)
brooklyn = pd.read_csv(fn2)
manhattan = pd.read_csv(fn3)
queens = pd.read_csv(fn4)
si = pd.read_csv(fn5)
paris = pd.read_csv(fn6)

In [74]:
pd.options.display.max_rows=5
bronx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 2448 non-null   int64  
 1   id                         2448 non-null   object 
 2   alias                      2448 non-null   object 
 3   name                       2448 non-null   object 
 4   image_url                  2429 non-null   object 
 5   is_closed                  2448 non-null   bool   
 6   url                        2448 non-null   object 
 7   review_count               2448 non-null   int64  
 8   categories                 2448 non-null   object 
 9   rating                     2448 non-null   float64
 10  transactions               2448 non-null   object 
 11  phone                      2289 non-null   float64
 12  display_phone              2289 non-null   object 
 13  distance                   2448 non-null   float

In [75]:
display(bronx.columns)
paris.columns

Index(['Unnamed: 0', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url',
       'review_count', 'categories', 'rating', 'transactions', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address',
       'price', 'total', 'region', 'Restaurant or Attraction?'],
      dtype='object')

Index(['Unnamed: 0', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url',
       'review_count', 'categories', 'rating', 'transactions', 'price',
       'phone', 'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address',
       'total', 'region', 'Restaurant or Attraction?'],
      dtype='object')

In [76]:
geo_groups = {'paris': paris, 'bronx': bronx, 'brooklyn': brooklyn, 'manhattan': manhattan, 'queens': queens, 'si': si}

In [77]:
nycp = pd.concat([bronx, brooklyn, manhattan, queens, si, paris])
print(nycp.name.count())

duplicates = nycp[nycp.duplicated(subset=['name', 'coordinates.latitude', 'coordinates.longitude'], keep=False)]
display(duplicates.head(1))

nycp = nycp.drop_duplicates(subset=['name', 'coordinates.latitude', 'coordinates.longitude'])
print(nycp.name.count())


19758


,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price,total,region,Restaurant or Attraction?
0,0,huCqalKTIWBv8w6fFsG49g,tobalá-the-bronx,Tobalá,https://s3-media4.fl.yelpcdn.com/bphoto/f2PBuC...,False,https://www.yelp.com/biz/tobal%C3%A1-the-bronx...,27,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,...,NaN,The Bronx,10463,US,NY,"['3732 Riverdale Ave', 'The Bronx, NY 10463']",NaN,2200,"{'center': {'longitude': -73.90588760375977, '...",Restaurant


4739


In [78]:
nycp.head(2)

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price,total,region,Restaurant or Attraction?
0,0,huCqalKTIWBv8w6fFsG49g,tobalá-the-bronx,Tobalá,https://s3-media4.fl.yelpcdn.com/bphoto/f2PBuC...,False,https://www.yelp.com/biz/tobal%C3%A1-the-bronx...,27,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,...,NaN,The Bronx,10463,US,NY,"['3732 Riverdale Ave', 'The Bronx, NY 10463']",NaN,2200,"{'center': {'longitude': -73.90588760375977, '...",Restaurant
1,1,I1AqEw-O6spamgtJLNPjBg,leche-y-miel-restaurant-bronx,Leche Y Miel Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/hVTUIn...,False,https://www.yelp.com/biz/leche-y-miel-restaura...,333,"[{'alias': 'comfortfood', 'title': 'Comfort Fo...",4.5,...,NaN,Bronx,10463,US,NY,"['5761 Broadway', 'Bronx, NY 10463']",$$,2200,"{'center': {'longitude': -73.90588760375977, '...",Restaurant


In [79]:
coords = {geo: df[['name', 'coordinates.latitude', 'coordinates.longitude']].
          drop_duplicates(subset=['name', 'coordinates.latitude', 'coordinates.longitude']) 
          for geo, df in geo_groups.items()}
coords['paris']
coords['manhattan']

,name,coordinates.latitude,coordinates.longitude
0,The Honey Well,40.828490,-73.948620
1,Harlem Public,40.828718,-73.948441
...,...,...,...
4285,Monty Hall,40.716398,-74.036372
4295,Accomplice The Show,40.725665,-73.997734


In [80]:
restaurants = {geo: df[df['Restaurant or Attraction?'] == 'Restaurant'][['name']] for geo, df in geo_groups.items()}
attractions = {geo: df[df['Restaurant or Attraction?'] != 'Restaurant'][['name']] for geo, df in geo_groups.items()}

for geo, df in restaurants.items():
    print(f"There are {restaurants[geo].count()[0]} restaurants in {geo}.")
print('\n')
for geo, df in attractions.items():
    print(f"There are {attractions[geo].count()[0]} attractions in {geo}.")

    
display(attractions['manhattan'])
display(restaurants['paris'])

There are 1050 restaurants in paris.
There are 1250 restaurants in bronx.
There are 1850 restaurants in brooklyn.
There are 2150 restaurants in manhattan.
There are 3043 restaurants in queens.
There are 600 restaurants in si.


There are 1050 attractions in paris.
There are 1198 attractions in bronx.
There are 1807 attractions in brooklyn.
There are 2150 attractions in manhattan.
There are 3010 attractions in queens.
There are 600 attractions in si.


,name
2150,Sister's Uptown Bookstore
2151,Universal Hip Hop Museum
...,...
4298,Poets House
4299,Angel Eye Astrology


,name
0,Le Comptoir de la Gastronomie
1,Bistro des Augustins
...,...
1048,Les Pères Populaires
1049,L'Ange 20


In [81]:
pd.options.display.max_rows=10
ratings=nycp[['rating']].drop_duplicates().sort_values(by=['rating'])
ratings

,rating
1293,0.0
1282,1.0
1259,1.5
1365,2.0
278,2.5
119,3.0
38,3.5
3,4.0
0,4.5
7,5.0


In [82]:
import re

display(nycp.id.count())

excluded_categories = ['Alternative','Book','Coach','Store','Grocer''Cosmetic','Psychic','Supernatural','Astrology','Palm','Reader','Reading','Fortune','Tarot','Medium','Reiki','Massage','Graffiti'] 
pattern = '|'.join(map(re.escape, excluded_categories))
nycp_norm = nycp[~nycp.categories.str.contains(pattern)]
nycp_norm = nycp[~nycp.name.str.contains(pattern)]

nycp_norm.id.count()

4739

4661

In [83]:
excluded_categories = ['Alternative', 'Book', 'Coach', 'Store', 'Grocer', 'Cosmetic', 'Psychic', 'Supernatural', 'Astrology', 'Palm', 'Reader', 'Reading', 'Fortune', 'Tarot', 'Medium', 'Reiki', 'Massage','Graffiti']
pattern = '|'.join(map(re.escape, excluded_categories))

nycp_filtered = {geo: df[~(df.categories.str.contains(pattern) | df.name.str.contains(pattern))] for geo, df in geo_groups.items()}

top_rated = {geo: df[df['rating'] >= 4][['name', 'rating']].drop_duplicates() for geo, df in nycp_filtered.items()}
# top_rated = {geo: df.drop_duplicates(subset='name')[df['rating'] >= 4][['name', 'rating']] for geo, df in nycp_filtered.items()}

top_rated['manhattan'].value_counts()
# .drop_duplicates(subset='name').value_counts()


name                    rating
11 Tigers               4.0       1
Philip Klipper          5.0       1
Piatto                  4.0       1
Piccola Cucina Osteria  4.5       1
Piccola Cucina Uptown   4.5       1
                                 ..
Harlem Nights Bar       4.0       1
Harlem Public           4.0       1
Harlem Seafood Soul     4.0       1
Harlem Stage Gatehouse  4.5       1
yasouvlaki              4.0       1
Length: 990, dtype: int64

In [84]:
top_rated_loc = {geo: pd.merge(top_rated[geo], coords[geo], on='name') for geo in geo_groups.keys()}
top_rated_loc['paris']
# .value_counts()


,name,rating,coordinates.latitude,coordinates.longitude
0,Le Comptoir de la Gastronomie,4.5,48.864516,2.345402
1,Bistro des Augustins,4.5,48.854754,2.342119
2,Les Antiquaires,4.5,48.858066,2.328237
3,L'Avant Comptoir,4.5,48.852020,2.338800
4,La Cordonnerie,4.5,48.865430,2.332370
...,...,...,...,...
731,Zorba,4.0,48.872000,2.376750
732,Salle principale,4.0,48.889387,2.384225
733,Le Bizart,4.5,48.851250,2.389920
734,La Comédia,4.0,48.857790,2.388860


In [85]:
%store top_rated_loc

Stored 'top_rated_loc' (dict)
